In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 

import warnings
warnings.filterwarnings('ignore') #slicing errors

In [2]:
model_2020 = pd.read_csv('./data_frames/2020_end_game_boxscores.csv')

In [3]:
model_2020.head()

,team,home_status,goals,shots,blocked_shots,penalty_minutes,power_play_percentage,power_play_goals,power_play_chances,faceoff_percent,takeaways,giveaways,hits,goal_diff,won
0,Pittsburgh Penguins,away,3,34,11,6,33.3,1.0,3.0,42.0,8,10,23,-3,no
1,Philadelphia Flyers,home,6,27,13,6,66.7,2.0,3.0,58.0,6,10,31,3,yes
2,Chicago Blackhawks,away,1,23,7,8,33.3,1.0,3.0,49.1,4,1,14,-4,no
3,Tampa Bay Lightning,home,5,33,12,6,50.0,2.0,4.0,50.9,4,1,16,4,yes
4,Montréal Canadiens,away,4,32,22,13,66.7,2.0,3.0,46.0,6,13,32,-1,no


Adding game IDs to the box scores model. 

In [4]:
game_id_list = []
for i in range(1,10):
    game_id_list.append(int(f'202002000{i}'))
    game_id_list.append(int(f'202002000{i}'))
for i in range(10,100):
    game_id_list.append(int(f'20200200{i}'))
    game_id_list.append(int(f'20200200{i}'))
for i in range(100,869):
    game_id_list.append(int(f'2020020{i}'))
    game_id_list.append(int(f'2020020{i}'))    

In [5]:
model_2020['game_ids'] = game_id_list

In [6]:
model_2020

,team,home_status,goals,shots,blocked_shots,penalty_minutes,power_play_percentage,power_play_goals,power_play_chances,faceoff_percent,takeaways,giveaways,hits,goal_diff,won,game_ids
0,Pittsburgh Penguins,away,3,34,11,6,33.3,1.0,3.0,42.0,8,10,23,-3,no,2020020001
1,Philadelphia Flyers,home,6,27,13,6,66.7,2.0,3.0,58.0,6,10,31,3,yes,2020020001
2,Chicago Blackhawks,away,1,23,7,8,33.3,1.0,3.0,49.1,4,1,14,-4,no,2020020002
3,Tampa Bay Lightning,home,5,33,12,6,50.0,2.0,4.0,50.9,4,1,16,4,yes,2020020002
4,Montréal Canadiens,away,4,32,22,13,66.7,2.0,3.0,46.0,6,13,32,-1,no,2020020003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1731,Vegas Golden Knights,home,4,29,17,10,25.0,1.0,4.0,42.9,9,2,25,3,yes,2020020866
1732,Colorado Avalanche,away,3,30,13,4,33.3,1.0,3.0,42.2,5,9,30,1,yes,2020020867
1733,Los Angeles Kings,home,2,18,12,6,0.0,0.0,2.0,57.8,1,7,23,-1,no,2020020867
1734,Arizona Coyotes,away,5,32,8,6,50.0,1.0,2.0,51.6,6,9,13,1,yes,2020020868


In [7]:
all_game_events = pd.read_csv('./data_frames/all_game_events_2020.csv')

In [8]:
all_game_events.drop(columns=['Unnamed: 0', 'score'], inplace=True)

In [9]:
all_game_events.shape

(264311, 8)

In [10]:
all_game_events.tail(5)

,event,team,x_coordinate,y_coordinate,period,period_time,game_id,home_team
264306,Goal,Arizona Coyotes,76.0,-15.0,4,02:30,2020020868,San Jose Sharks
264307,Period End,NaN,NaN,NaN,4,02:30,2020020868,San Jose Sharks
264308,Period Official,NaN,NaN,NaN,4,02:30,2020020868,San Jose Sharks
264309,Game End,NaN,NaN,NaN,4,02:30,2020020868,San Jose Sharks
264310,Game Official,NaN,NaN,NaN,4,02:30,2020020868,San Jose Sharks


In [11]:
all_game_events.shape

(264311, 8)

In [12]:
all_game_events['event'].value_counts()

Faceoff                     48837
Shot                        47658
Hit                         38783
Stoppage                    37823
Blocked Shot                22763
Missed Shot                 19533
Giveaway                    15234
Takeaway                    10618
Penalty                      6065
Goal                         5222
Period End                   2892
Period Official              2892
Period Start                 2040
Period Ready                 2019
Game End                      876
Game Official                 869
Official Challenge            108
Shootout Complete              75
Emergency Goaltender            2
Early Intermission Start        1
Early Intermission End          1
Name: event, dtype: int64

engineered features: ice tilt

Need to inverse the x coordinate numbers for the second period, since the teams change sides but the coordinates remain the same. 

In [21]:
#https://datascience.stackexchange.com/questions/56668/pandas-change-value-of-a-column-based-another-column-condition

In [31]:
game_id_list_single = []
for i in range(1,10):
    game_id_list_single.append(int(f'202002000{i}'))
for i in range(10,100):
    game_id_list_single.append(int(f'20200200{i}'))
for i in range(100,869):
    game_id_list_single.append(int(f'2020020{i}')) 

In [32]:
# function that creates the ice tilt for games within a list of game ID's. 
# code/inspiration for the mask #https://datascience.stackexchange.com/questions/56668/pandas-change-value-of-a-column-based-another-column-condition
def ice_tilt(game_ids):
    ice_tilt_list = []
    for i in game_ids:
        game_df = all_game_events[all_game_events['game_id'] == i].copy()
        game_ice_tilt = game_df[(game_df['event'] == 'Faceoff') | (game_df['event'] == 'Shot') | (game_df['event'] == 'Hit') | (game_df['event'] == 'Missed Shot')]                
        mask = (game_ice_tilt['period']==2)
        game_ice_tilt['x_coordinate'][mask] = game_ice_tilt['x_coordinate'] * -1
        tilt = game_ice_tilt['x_coordinate'].sum()
        ice_tilt_list.append(tilt)
    return ice_tilt_list

In [33]:
full_tilt = ice_tilt(game_id_list_single)

Get the differences in the stats for each game, able to use the data frame structure here to complete this task easily. 

In [42]:
game_difference_df = pd.DataFrame()
a=0
for i in range(1,1736,2):
        game_info = model_2020.iloc[i][13:].to_dict()
        game_stats = model_2020.iloc[i][2:13] -  model_2020.iloc[i-1][2:13]
        game_info.update(game_stats.to_dict())
        temp_diff = pd.DataFrame(game_info, index=[a])
        game_difference_df = pd.concat([game_difference_df, temp_diff], axis=0, ignore_index=True)
        a += 1    

In [43]:
game_difference_df.head()

,goal_diff,won,game_ids,goals,shots,blocked_shots,penalty_minutes,power_play_percentage,power_play_goals,power_play_chances,faceoff_percent,takeaways,giveaways,hits
0,3,yes,2020020001,3,-7,2,0,33.4,1.0,0.0,16.0,-2,0,8
1,4,yes,2020020002,4,10,5,-2,16.7,1.0,1.0,1.8,0,0,2
2,1,yes,2020020003,1,2,-5,-2,-16.7,0.0,1.0,8.0,-1,-4,-18
3,-2,no,2020020004,-2,-4,3,-4,0.0,0.0,2.0,3.6,-1,4,-1
4,-3,no,2020020005,-3,-5,-8,-4,25.0,1.0,2.0,8.8,3,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
863,2,yes,2020020864,2,-16,2,0,0.0,0.0,0.0,-5.0,-3,0,-9
864,1,yes,2020020865,1,3,7,-6,25.0,1.0,3.0,-1.8,7,3,1
865,3,yes,2020020866,3,8,-1,-2,25.0,1.0,1.0,-14.2,0,-9,-17
866,-1,no,2020020867,-1,-12,-1,2,-33.3,-1.0,-1.0,15.6,-4,-2,-7


Need to invert the ice tilt, in order to make it positive for the home team.

In [44]:
full_tilt = [-1 * x for x in full_tilt]

In [45]:
game_difference_df['ice_tilt'] = full_tilt

In [46]:
game_difference_df.head()

,goal_diff,won,game_ids,goals,shots,blocked_shots,penalty_minutes,power_play_percentage,power_play_goals,power_play_chances,faceoff_percent,takeaways,giveaways,hits,ice_tilt
0,3,yes,2020020001,3,-7,2,0,33.4,1.0,0.0,16.0,-2,0,8,-1058.0
1,4,yes,2020020002,4,10,5,-2,16.7,1.0,1.0,1.8,0,0,2,89.0
2,1,yes,2020020003,1,2,-5,-2,-16.7,0.0,1.0,8.0,-1,-4,-18,1328.0
3,-2,no,2020020004,-2,-4,3,-4,0.0,0.0,2.0,3.6,-1,4,-1,695.0
4,-3,no,2020020005,-3,-5,-8,-4,25.0,1.0,2.0,8.8,3,5,1,405.0


In [47]:
#game_difference_df.to_csv('./data_frames/game_stats_differences_2020.csv', index=False)

In [48]:
game_difference_df['won'].value_counts()

no     436
yes    432
Name: won, dtype: int64

This will be the main modeling data frame for training on data frame differences of full game stats. It is based off of home team stats differentials. positive numbers mean the home team has more stats in the value, and negative means they are trailing in that stat. 

multiple model approach DFs

Creating models for each time step during a game. This will be a dataframe that has the differences in stats at 2 minute intervals for the entire season of 2020-2021. The first data frame will be the stats difference for every game 2 minutes in, and the next one 4 minutes in until the game is over. 

In [52]:
#turn the period time column, which is a string into an int so we can filter on it. 
def time_to_int(column):
    output_list = []
    for i in [x for x in column]:
        numeric_time = []
        for digit in i:
            if digit != ":":
                numeric_time.append((digit))
            if len(numeric_time) == 4:
                numeric_time = "".join(numeric_time)
                int(numeric_time)
        output_list.append(numeric_time)  
    output_list = [int(x) for x in output_list]
    return output_list
        

In [55]:
# adjust the period time column as an int, so that it has the total game time, not just the period time. 
def period_time_to_full_time(df):
    first_period = df[df['period'] == 1]
    second_period = df[df['period'] == 2]
    third_period = df[df['period'] == 3]
    second_period['period_time_int'] = second_period['period_time_int'] + 2000
    third_period['period_time_int'] = third_period['period_time_int'] + 4000
    output_df = pd.concat([first_period,second_period], axis=0, ignore_index=True)
    output_df = pd.concat([output_df,third_period], axis=0, ignore_index=True)
    return output_df
    

In [65]:
# function that returns a two column data frame for the given time interval. the home team stats and 
# away team stats 
def minutes_into_game_df(df, minute_interval = 200):
    events = df[df['period_time_int'] <= minute_interval]

    events_home = events[events['team'] == events['home_team']]
    events_home_dict = events_home.groupby('event').count()['team'].to_dict()
    ice_tilt_minutes = events_home.groupby('event')['x_coordinate'].sum().sum()
    base_dic_home = {
    'Goal':0, 'Shot':0, 'Blocked Shot': 0,
    'Faceoff':0 , 'Takeaway':0, 'Giveaway': 0,
    'Hit': 0, 'Missed Shot':0, 'Penalty': 0, 'Stoppage':0
    }
    base_dic_home.update(events_home_dict)

    events_away = events[events['team'] != events['home_team']]
    events_away_dict = events_away.groupby('event').count()['team'].to_dict()
    ice_tilt_minutes = events_away.groupby('event')['x_coordinate'].sum().sum()
    base_dic_away = {
    'Goal':0, 'Shot':0, 'Blocked Shot': 0,
    'Faceoff':0 , 'Takeaway':0, 'Giveaway': 0,
    'Hit': 0, 'Missed Shot':0, 'Penalty': 0, 'Stoppage':0
    }
    base_dic_away.update(events_away_dict)
    output_df = pd.DataFrame(base_dic_away, index=[0])
    concat_df = pd.DataFrame(base_dic_home, index=[1])
    output_df = pd.concat([output_df,concat_df], axis=0, ignore_index=True)
    return output_df


In [68]:
two_minutes_in_2020 = pd.DataFrame()
for i in game_id_list_single:
    game_df = all_game_events[all_game_events['game_id'] == i].copy()
    game_df['period_time_int'] = time_to_int(game_df['period_time'])
    game_df = period_time_to_full_time(game_df)
    temp_df = minutes_into_game_df(game_df)
    two_minutes_in_2020 = pd.concat([two_minutes_in_2020,temp_df], axis=0, ignore_index=True)

In [69]:
# a function that uses the previous function to get all the game events at a certain time interval
# and transform it into a modeling dataframe with target variable over a list of game ids. 
def minutes_into_game_modeling_frame(game_ids, time_interval):
    output_df = pd.DataFrame()
    for n in game_ids:
        game_df = all_game_events[all_game_events['game_id'] == n].copy()
        game_df['period_time_int'] = time_to_int(game_df['period_time'])
        game_df = period_time_to_full_time(game_df)
        game_df = game_df[(game_df['event'] == 'Shot') | (game_df['event'] == 'Goal') | 
                          (game_df['event'] == 'Blocked Shot')| (game_df['event'] == 'Faceoff') |
                          (game_df['event'] == 'Takeaway') | (game_df['event'] == 'Giveaway') |
                          (game_df['event'] == 'Hit') | (game_df['event'] == 'Missed Shot') |
                          (game_df['event'] == 'Penalty')]
        temp_df = minutes_into_game_df(game_df, time_interval)
        output_df = pd.concat([output_df, temp_df], axis=0, ignore_index=True)
    output_df['won'] = model_2020['won']
    output_df.drop(columns='Stoppage', inplace=True)
    return output_df

In [ ]:
# loop that creates, and then saves data frames for minute intervals into game, and takes the
# difference so it can be modeled on. 
counter = 200
while counter <=  6000:
    counter_df = minutes_into_game_modeling_frame(game_id_list_single,counter)
    differences_df = pd.DataFrame()
    for i in range(1,1737,2):
        winner = counter_df.iloc[i][9]
        game_stats = counter_df.iloc[i][:9] - counter_df.iloc[i-1][:9]
        temp_df = pd.DataFrame(game_stats.to_dict(), index=[i])
        temp_df['won'] = winner
        differences_df = pd.concat([differences_df, temp_df], axis=0, ignore_index=True)
             
    differences_df.to_csv(f'./data_frames/{counter}_minutes_diffs_2020.csv', index = False)
    counter_df.to_csv(f'./data_frames/{counter}_minutes_2020.csv', index = False)
    counter += 200